In [1]:
%matplotlib inline

In [2]:
from IPython.display import clear_output
from pathlib import Path
from PIL import Image
from tqdm import tqdm

import glob
import os
import pprint
import random
import shutil
import tarfile
import time
import torch
import torchvision

import matplotlib.pyplot as plt
import numpy as np
import onnxruntime as ort
import torch.nn.functional as F
import torchvision.transforms as T

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_model(model, path):
    
    ckpt_dict = torch.load(path)
    
    model.load_state_dict(ckpt_dict['model_state_dict'])
    
    return ckpt_dict

In [14]:
def save_to_onnx(model, input_shape, path=None):
    bs, c, h, w = input_shape
    
    dummy_input = torch.randn(bs, c, h, w, device='cuda')
    
    model.to('cuda')
    
    input_names = [ "input" ]
    output_names = [ "output" ]
    
    if not path:
        path = model.__class__.__name__ + '.onnx'
    
    torch.onnx.export(model, dummy_input, path, verbose=True, input_names=input_names, output_names=output_names)
    
    return path

In [4]:
device = torch.device('cpu')

In [5]:
train_ds = torchvision.datasets.ImageFolder(root='/workspace/data/LAICC_2023/training/', transform=None,
                                            target_transform=None, loader=Image.open,
                                            is_valid_file=None)

In [6]:
model = torchvision.models.resnet18(weights=None)

In [7]:
model.fc = torch.nn.Linear(model.fc.in_features, len(train_ds.classes))

In [8]:
for param in model.parameters():
    param.requires_grad = False

In [9]:
chkpt_path = os.path.join('/workspace/models',
                          os.listdir('/workspace/models')[-1])

In [10]:
chkpt_dict = load_model(model, chkpt_path)

{'epoch': 19,
 'model_state_dict': OrderedDict([('conv1.weight',
               tensor([[[[ 2.8641e-02,  3.5155e-02,  4.5284e-02,  ...,  1.0424e-01,
                           6.4021e-02,  1.6429e-02],
                         [ 7.4465e-02,  8.1451e-02, -3.4816e-02,  ..., -1.7951e-01,
                          -4.2363e-02,  6.9026e-02],
                         [ 5.1826e-02,  1.3132e-01,  3.6952e-01,  ...,  5.9980e-01,
                           3.3619e-01,  1.3055e-01],
                         ...,
                         [ 1.7361e-02,  5.8674e-02,  1.1145e-01,  ..., -3.0507e-01,
                          -3.9158e-01, -2.3388e-01],
                         [ 8.6305e-02,  1.0920e-01,  1.4159e-01,  ...,  4.8709e-01,
                           4.5520e-01,  2.1168e-01],
                         [ 4.7608e-02,  7.1317e-02,  6.0750e-02,  ..., -5.0018e-02,
                           7.6857e-03,  5.5796e-02]],
               
                        [[ 3.7672e-03, -1.1274e-02, -1.1464e-02,  

In [11]:
model.to('cpu')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
# IF you are satisfied with your models performance, save to ONNX
# You need to change your input shape. It should be of the form
# (batch_size, num_channels, height, width)
# You will get height and width from the transform
# T.Resize(size=(224, 224)), which would mean we would have
input_shape = (10, 3, 224, 224)


path = f'/workspace/models/{model.__class__.__name__}18_{device}_{int(time.time())}.onnx'

save_to_onnx(model, input_shape, path=path)

Exported graph: graph(%input : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cuda:0),
      %fc.weight : Float(7, 512, strides=[512, 1], requires_grad=0, device=cuda:0),
      %fc.bias : Float(7, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_193 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_194 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_196 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_197 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_199 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_200 : Float(64, strides=[1], requires_grad=0, device=cuda:0),
      %onnx::Conv_202 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cuda:0),
      %onnx::Conv_203 : Float(64, strides=[1], requires_grad=0, device=cuda:0),


'/workspace/models/ResNet18_cpu_1688054954.onnx'